# Agregación y Agrupación

Una parte esencial del análisis de grandes datos es el resumen eficiente: computar agregaciones como ``sum()``, ``mean()``, ``median()``, ``min()`` y ``max()``, en el que un solo número da una idea de la naturaleza de un conjunto de datos potencialmente grande. En esta sección, exploraremos agregaciones en Pandas, desde operaciones simples similares a las que hemos visto en matrices NumPy, hasta operaciones más sofisticadas basadas en el concepto de un ``groupby``.

Por conveniencia, usaremos la misma función ``display`` mágica que hemos visto en secciones anteriores:

In [ ]:
import numpy as np
import pandas as pd

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

## Datos de planetas

Aquí usaremos el conjunto de datos Planets, disponible a través del paquete Seaborn. Brinda información sobre los planetas que los astrónomos han descubierto alrededor de otras estrellas (conocidos como planetas extrasolares o exoplanetas para abreviar). Se puede descargar con un simple comando de Seaborn: 

In [ ]:
import seaborn as sns
planets = sns.load_dataset('planets')
planets.shape

(1035, 6)

In [ ]:
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


Esto tiene algunos detalles sobre los más de 1000 planetas extrasolares descubiertos hasta 2014.

## Agregación Simple en Pandas

Anteriormente, exploramos algunas de las agregaciones de datos disponibles para matrices NumPy. Al igual que con una matriz NumPy unidimensional, para Pandas ``Series`` los agregados devuelven un solo valor:

In [ ]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.rand(5))
ser

0    0.374540
1    0.950714
2    0.731994
3    0.598658
4    0.156019
dtype: float64

In [ ]:
ser.sum()

2.8119254917081569

In [ ]:
ser.mean()

0.56238509834163142

Para ``DataFrame``, de forma predeterminada, los agregados devuelven resultados dentro de cada columna:

In [ ]:
df = pd.DataFrame({'A': rng.rand(5),
                   'B': rng.rand(5)})
df

,A,B
0,0.155995,0.020584
1,0.058084,0.969910
2,0.866176,0.832443
3,0.601115,0.212339
4,0.708073,0.181825


In [ ]:
df.mean()

A    0.477888
B    0.443420
dtype: float64

Al especificar el argumento ``axis``, en su lugar puede agregar dentro de cada fila:

In [ ]:
df.mean(axis='columns')

0    0.088290
1    0.513997
2    0.849309
3    0.406727
4    0.444949
dtype: float64

Pandas ``Series`` y ``DataFrames`` tienen un método de conveniencia ``describe()`` que calcula varios agregados comunes para cada columna y devuelve el resultado. Usemos esto en los datos de los Planetas, por ahora soltando filas con valores faltantes:

In [ ]:
planets.dropna().describe()

,number,orbital_period,mass,distance,year
count,498.00000,498.000000,498.000000,498.000000,498.000000
mean,1.73494,835.778671,2.509320,52.068213,2007.377510
std,1.17572,1469.128259,3.636274,46.596041,4.167284
min,1.00000,1.328300,0.003600,1.350000,1989.000000
25%,1.00000,38.272250,0.212500,24.497500,2005.000000
50%,1.00000,357.000000,1.245000,39.940000,2009.000000
75%,2.00000,999.600000,2.867500,59.332500,2011.000000
max,6.00000,17337.500000,25.000000,354.000000,2014.000000


Esta puede ser una forma útil de comenzar a comprender las propiedades generales de un conjunto de datos. Por ejemplo, vemos en la columna ``year`` que, aunque los exoplanetas se descubrieron ya en 1989, la mitad de todos los exoplanetas conocidos no se descubrieron hasta 2010 o después. Esto se debe en gran parte a la Kepler , que es un telescopio espacial diseñado específicamente para encontrar planetas eclipsantes alrededor de otras estrellas.

La siguiente tabla resume algunas otras agregaciones integradas de Pandas: 

| Aggregation              | Description                     |
|--------------------------|---------------------------------|
| ``count()``              | Total number of items           |
| ``first()``, ``last()``  | First and last item             |
| ``mean()``, ``median()`` | Mean and median                 |
| ``min()``, ``max()``     | Minimum and maximum             |
| ``std()``, ``var()``     | Standard deviation and variance |
| ``mad()``                | Mean absolute deviation         |
| ``prod()``               | Product of all items            |
| ``sum()``                | Sum of all items                |

Todos estos son métodos de objetos ``DataFrame`` y ``Series``.

Sin embargo, para profundizar en los datos, los agregados simples a menudo no son suficientes. El siguiente nivel de resumen de datos es el ``groupby`` operación, que le permite calcular agregados en subconjuntos de datos de manera rápida y eficiente.

## GroupBy: Split, Apply, Combine

Las agregaciones simples pueden darle una idea de su conjunto de datos, pero a menudo preferiríamos agregar condicionalmente en alguna etiqueta o índice: esto se implementa en el llamado la operación ``groupby``. El nombre "agrupar por" proviene de un comando en el lenguaje de la base de datos SQL, pero quizás sea más esclarecedor pensar en él en los términos acuñados por primera vez por Hadley Wickham de Rstats: dividir, aplicar, combinar. 

### Split, apply, combine

Un ejemplo canónico de esta operación dividir-aplicar-combinar, donde "aplicar" es una agregación de suma, se ilustra en esta figura: 

![](./figures/03.08-split-apply-combine.png)
[figure source in Appendix](06.00-Figure-Code.ipynb#Split-Apply-Combine)

Esto deja claro que es lo que se logra cocn ``groupby``:

    El *split* consiste en dividir y agrupar un ``DataFrame`` dependiendo del valor de la clave especificada.
    El *apply* implica calcular alguna función, generalmente un agregado, transformación o filtrado, dentro de los grupos individuales.
    El *combine* combina los resultados de estas operaciones en una matriz de salida.


Si bien esto ciertamente podría hacerse manualmente usando alguna combinación de los comandos de enmascaramiento, agregación y fusión cubiertos anteriormente, una comprensión importante es que las divisiones intermedias no necesitan instanciarse explícitamente . Más bien, el ``GroupBy`` puede (a menudo) hacer esto en un solo paso sobre los datos, actualizando la suma, la media, el conteo, el mínimo u otro agregado para cada grupo en el camino. El poder de ``GroupBy`` es que abstrae estos pasos: el usuario no necesita pensar en cómo se realiza el cálculo bajo el capó, sino que piensa en la operación como un todo . 

Como ejemplo concreto, echemos un vistazo al uso de Pandas para el cálculo que se muestra en este diagrama. Comenzaremos creando la entrada ``DataFrame``:

In [ ]:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data': range(6)}, columns=['key', 'data'])
df

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


La operación más básica de división-aplicación-combinación se puede calcular con la ``groupby()`` método de ``DataFrames``, pasando el nombre de la columna clave deseada:

In [ ]:
df.groupby('key')

Observe que lo que se devuelve no es un conjunto de ``DataFrames``, pero un objeto ``DataFrameGroupBy``. Este objeto es donde está la magia: puedes pensar en él como una vista especial del ``DataFrame``, que está preparado para profundizar en los grupos, pero no realiza ningún cálculo real hasta que se aplica la agregación. Este enfoque de "evaluación perezosa" significa que los agregados comunes se pueden implementar de manera muy eficiente de una manera que es casi transparente para el usuario. 

Para producir un resultado, podemos aplicar un agregado a este objeto ``DataFrameGroupBy``, que realizará los pasos apropiados de aplicar/combinar para producir el resultado deseado: 

In [ ]:
df.groupby('key').sum()

,data
key,
A,3
B,5
C,7


El método ``sum()`` es solo una posibilidad aquí; puede aplicar prácticamente cualquier función de agregación común de Pandas o NumPy, así como prácticamente cualquier función válida ``DataFrame`` operación, como veremos en la siguiente discusión.

### The GroupBy object

El objeto ``GroupBy`` es una abstracción muy flexible. En muchos sentidos, simplemente puede tratarlo como si fuera una colección de ``DataFrames``, y hace las cosas difíciles debajo del capó. Veamos algunos ejemplos usando los datos de los Planetas. 

Quizá las operaciones más importantes que ofrece un ``GroupBy`` son *aggregate*, *filter*, *transform* y *apply*. Presentemos algunas de las otras funciones que se pueden usar con la operación básica ``GroupBy``. 

#### Indización de columnas

El objeto ``GroupBy`` admite la indexación de columnas de la misma manera que el ``DataFrame`` y devuelve un objeto modificado ``GroupBy``. Por ejemplo:

In [ ]:
planets.groupby('method')

In [ ]:
planets.groupby('method')['orbital_period']

Aquí hemos seleccionado un particular ``Series`` grupo del original ``DataFrame`` grupo por referencia a su nombre de columna. Al igual que con el objeto ``GroupBy``, no se realiza ningún cálculo hasta que llamamos algún agregado en el objeto:

In [ ]:
planets.groupby('method')['orbital_period'].median()

method
Astrometry                         631.180000
Eclipse Timing Variations         4343.500000
Imaging                          27500.000000
Microlensing                      3300.000000
Orbital Brightness Modulation        0.342887
Pulsar Timing                       66.541900
Pulsation Timing Variations       1170.000000
Radial Velocity                    360.200000
Transit                              5.714932
Transit Timing Variations           57.011000
Name: orbital_period, dtype: float64

Esto da una idea de la escala general de períodos orbitales (en días) a los que es sensible cada método.

#### Iteration over groups

El objeto ``GroupBy`` admite la iteración directa sobre los grupos, devolviendo cada grupo como un ``Series`` o ``DataFrame``: 

In [ ]:
for (method, group) in planets.groupby('method'):
    print("{0:30s} shape={1}".format(method, group.shape))

Astrometry                     shape=(2, 6)
Eclipse Timing Variations      shape=(9, 6)
Imaging                        shape=(38, 6)
Microlensing                   shape=(23, 6)
Orbital Brightness Modulation  shape=(3, 6)
Pulsar Timing                  shape=(5, 6)
Pulsation Timing Variations    shape=(1, 6)
Radial Velocity                shape=(553, 6)
Transit                        shape=(397, 6)
Transit Timing Variations      shape=(4, 6)


Esto puede ser útil para hacer ciertas cosas manualmente, aunque a menudo es mucho más rápido usar la funcionalidad ``apply``, que discutiremos momentáneamente.

#### Métodos de envío

A través de alguna clase mágica de Python, cualquier método no implementado explícitamente por el objeto ``GroupBy`` se pasará a través y se llamará a los grupos, ya sea que estén ``DataFrame``o ``Series``. Por ejemplo, puede utilizar el método ``describe()`` de ``DataFrames`` para realizar un conjunto de agregaciones que describen cada grupo en los datos: 

In [ ]:
planets.groupby('method')['year'].describe().unstack()

,count,mean,std,min,25%,50%,75%,max
method,,,,,,,,
Astrometry,2.0,2011.500000,2.121320,2010.0,2010.75,2011.5,2012.25,2013.0
Eclipse Timing Variations,9.0,2010.000000,1.414214,2008.0,2009.00,2010.0,2011.00,2012.0
Imaging,38.0,2009.131579,2.781901,2004.0,2008.00,2009.0,2011.00,2013.0
Microlensing,23.0,2009.782609,2.859697,2004.0,2008.00,2010.0,2012.00,2013.0
Orbital Brightness Modulation,3.0,2011.666667,1.154701,2011.0,2011.00,2011.0,2012.00,2013.0
Pulsar Timing,5.0,1998.400000,8.384510,1992.0,1992.00,1994.0,2003.00,2011.0
Pulsation Timing Variations,1.0,2007.000000,NaN,2007.0,2007.00,2007.0,2007.00,2007.0
Radial Velocity,553.0,2007.518987,4.249052,1989.0,2005.00,2009.0,2011.00,2014.0
Transit,397.0,2011.236776,2.077867,2002.0,2010.00,2012.0,2013.00,2014.0


Mirar esta tabla nos ayuda a comprender mejor los datos: por ejemplo, la gran mayoría de los planetas han sido descubiertos por los métodos de Velocidad Radial y Tránsito, aunque este último solo se volvió común (debido a telescopios nuevos y más precisos) en la última década. . Los métodos más nuevos parecen ser la variación del tiempo de tránsito y la modulación del brillo orbital, que no se usaron para descubrir un nuevo planeta hasta 2011.

Este es solo un ejemplo de la utilidad de los métodos de despacho. Observe que se aplican a cada grupo individual y los resultados se combinan dentro de ``GroupBy`` y volvió. De nuevo, cualquier válido ``DataFrame/Series`` El método se puede utilizar en el correspondiente objeto ``GroupBy``, que permite algunas operaciones muy flexibles y potentes!

### Agregar, filtrar, transformar, aplicar

La discusión anterior se centró en la agregación para la operación combinada, pero hay más opciones disponibles. En particular, los objetos ``GroupBy`` tienen ``aggregate()``, ``filter()``, ``transform()`` y ``apply()`` métodos que implementan eficientemente una variedad de operaciones útiles antes de combinar los datos agrupados.

A los efectos de las siguientes subsecciones, utilizaremos este ``DataFrame``:

In [ ]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                   'data2': rng.randint(0, 10, 6)},
                   columns = ['key', 'data1', 'data2'])
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


#### Aggregation

Ahora estamos familiarizados con ``GroupBy`` agregaciones con ``sum()``, ``median()``, y similares, pero el método ``aggregate()`` permite aún más flexibilidad. Puede tomar una cadena, una función o una lista de las mismas y calcular todos los agregados a la vez. Aquí hay un ejemplo rápido que combina todo esto: 

In [ ]:
df.groupby('key').aggregate(['min', np.median, max])

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     3    4.0   5
B       1    2.5   4     0    3.5   7
C       2    3.5   5     3    6.0   9

Otro patrón útil es pasar los nombres de las columnas de asignación de un diccionario a las operaciones que se aplicarán en esa columna:

In [ ]:
df.groupby('key').aggregate({'data1': 'min',
                             'data2': 'max'})

,data1,data2
key,,
A,0,5
B,1,7
C,2,9


#### Filtering

Una operación de filtrado le permite colocar datos en función de las propiedades del grupo. Por ejemplo, podríamos querer mantener todos los grupos en los que la desviación estándar es mayor que algún valor crítico: 

In [ ]:
def filter_func(x):
    return x['data2'].std() > 4

display('df', "df.groupby('key').std()", "df.groupby('key').filter(filter_func)")

df
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

df.groupby('key').std()
       data1     data2
key                   
A    2.12132  1.414214
B    2.12132  4.949747
C    2.12132  4.242641

df.groupby('key').filter(filter_func)
  key  data1  data2
1   B      1      0
2   C      2      3
4   B      4      7
5   C      5      9

La función de filtro debe devolver un valor booleano que especifique si el grupo pasa el filtrado. Aquí, debido a que el grupo A no tiene una desviación estándar superior a 4, se elimina del resultado.

#### Transformation

Mientras que la agregación debe devolver una versión reducida de los datos, la transformación puede devolver alguna versión transformada de los datos completos para recombinarlos. Para tal transformación, la salida tiene la misma forma que la entrada. Un ejemplo común es centrar los datos restando la media del grupo: 

In [ ]:
df.groupby('key').transform(lambda x: x - x.mean())

,data1,data2
0,-1.5,1.0
1,-1.5,-3.5
2,-1.5,-3.0
3,1.5,-1.0
4,1.5,3.5
5,1.5,3.0


#### El metodo apply()

El método ``apply()`` le permite aplicar una función arbitraria a los resultados del grupo. La función debe tomar un ``DataFrame``, y devolver un objeto Pandas (por ejemplo, DataFrame, Series) o un escalar; la operación de combinación se adaptará al tipo de salida devuelta.

Por ejemplo, aquí hay un ``apply()`` que normaliza la primera columna por la suma de la segunda: 

In [ ]:
def norm_by_data2(x):
    # x is a DataFrame of group values
    x['data1'] /= x['data2'].sum()
    return x

display('df', "df.groupby('key').apply(norm_by_data2)")

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9
,key,data1,data2
0,A,0.000000,5
1,B,0.142857,0
2,C,0.166667,3


``apply()`` dentro de un ``GroupBy`` es bastante flexible: el único criterio es que la función tome un ``DataFrame`` y devuelve un objeto Pandas o escalar; ¡lo que hagas en el medio depende de ti!

### Especificando la clave dividida

En los ejemplos simples presentados antes, dividimos el ``DataFrame`` en un solo nombre de columna. Esta es solo una de las muchas opciones mediante las cuales se pueden definir los grupos, y aquí veremos algunas otras opciones para la especificación de grupos.

#### Una lista, matriz, serie o índice que proporciona las claves de agrupación

La clave puede ser cualquier serie o lista con una longitud que coincida con la del ``DataFrame``. Por ejemplo:

In [ ]:
L = [0, 1, 0, 1, 2, 0]
display('df', 'df.groupby(L).sum()')

df
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

df.groupby(L).sum()
   data1  data2
0      7     17
1      4      3
2      4      7

Por supuesto, esto significa que hay otra forma más detallada de lograr el ``df.groupby('key')`` desde antes:

In [ ]:
display('df', "df.groupby(df['key']).sum()")

df
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

df.groupby(df['key']).sum()
     data1  data2
key              
A        3      8
B        5      7
C        7     12

#### Un índice de mapeo de diccionario o serie para agrupar

Otro método es proporcionar un diccionario que asigne valores de índice a las claves de grupo:

In [ ]:
df2 = df.set_index('key')
mapping = {'A': 'vowel', 'B': 'consonant', 'C': 'consonant'}
display('df2', 'df2.groupby(mapping).sum()')

,data1,data2
key,,
A,0,5
B,1,0
C,2,3
A,3,3
B,4,7
C,5,9
,data1,data2
consonant,12,19
vowel,3,8


#### Cualquier función de Python

Similar a la asignación, puede pasar cualquier función de Python que ingrese el valor del índice y genere el grupo:


In [ ]:
display('df2', 'df2.groupby(str.lower).mean()')

,data1,data2
key,,
A,0,5
B,1,0
C,2,3
A,3,3
B,4,7
C,5,9
,data1,data2
a,1.5,4.0
b,2.5,3.5


#### Una lista de claves válidas

Además, cualquiera de las opciones clave anteriores se puede combinar para agrupar en un índice múltiple:

In [ ]:
df2.groupby([str.lower, mapping]).mean()

,,data1,data2
a,vowel,1.5,4.0
b,consonant,2.5,3.5
c,consonant,3.5,6.0


### Ejemplo de agrupación

Como ejemplo de esto, en un par de líneas de código Python podemos juntar todo esto y contar los planetas descubiertos por método y por década:

In [ ]:
decade = 10 * (planets['year'] // 10)
decade = decade.astype(str) + 's'
decade.name = 'decade'
planets.groupby(['method', decade])['number'].sum().unstack().fillna(0)

decade,1980s,1990s,2000s,2010s
method,,,,
Astrometry,0.0,0.0,0.0,2.0
Eclipse Timing Variations,0.0,0.0,5.0,10.0
Imaging,0.0,0.0,29.0,21.0
Microlensing,0.0,0.0,12.0,15.0
Orbital Brightness Modulation,0.0,0.0,0.0,5.0
Pulsar Timing,0.0,9.0,1.0,1.0
Pulsation Timing Variations,0.0,0.0,1.0,0.0
Radial Velocity,1.0,52.0,475.0,424.0
Transit,0.0,0.0,64.0,712.0


Esto muestra el poder de combinar muchas de las operaciones que hemos discutido hasta este punto al observar conjuntos de datos realistas. ¡Inmediatamente obtenemos una comprensión aproximada de cuándo y cómo se han descubierto los planetas en las últimas décadas! 

Aquí sugeriría profundizar en estas pocas líneas de código y evaluar los pasos individuales para asegurarse de que comprende exactamente lo que están haciendo con el resultado. Ciertamente es un ejemplo algo complicado, pero comprender estas piezas le dará los medios para explorar de manera similar sus propios datos. 